In [2]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [47]:
# import new_merge.csv
df = pd.read_csv(r'.\intermediate\new_merge.csv')

In [48]:
# import soil
soil = pd.read_csv(r'.\intermediate\soil_moisture_0_to_7cm.csv')

In [49]:
# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
soil['date'] = pd.to_datetime(soil['date'])

In [50]:
# rename lat_x as lat and lon_x as lon
df.rename(columns={'lat_x': 'lat', 'lon_x': 'lon'}, inplace=True)

In [51]:
# check if date lat lon form a unique key
print(df.groupby(['date', 'lat', 'lon']).size().max())
print(soil.groupby(['date', 'lat', 'lon']).size().max())

1
1


In [52]:
# merge df and soil, left on date, lat, lon, add indicator
df = df.merge(soil, how='left', on=['date', 'lat', 'lon'], indicator=True)

In [53]:
# see merge result
print(df.groupby('_merge').size())

_merge
left_only         0
right_only        0
both          38776
dtype: int64


In [54]:
# drop _merge
df.drop('_merge', axis=1, inplace=True)

In [12]:
# output to csv
df.to_csv('merge_complete_soil.csv', index=False)

In [13]:
# extract unique lat lon
lat_lon = df[['lat', 'lon']].drop_duplicates()

In [14]:
lat_lon

,lat,lon
0,-5.533893,14.070903
1,-5.493558,14.068317
2,-5.289988,14.314904
3,6.765411,15.778695
4,7.248169,16.437358
...,...,...
37994,-13.463138,38.398858
38002,-13.465048,38.406667
38150,3.426228,46.034461
38483,-13.570206,33.788265


In [37]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import tqdm
import time

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session



# create a new df to store the data
result = pd.DataFrame(columns=['lat', 'lon', 'elevation'])

# loop through each lat and lon
for i in tqdm.tqdm(range(len(lat_lon))):
    # request the data from the API
    url = 'https://api.open-meteo.com/v1/elevation?latitude=' + str(lat_lon['lat'].iloc[i]) + '&longitude=' + str(lat_lon['lon'].iloc[i])
    try:
        response = requests_retry_session().get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    data = response.json()
    # create a new df to store the data
    df = pd.DataFrame(columns=['lat', 'lon', 'elevation'])
    # extract time and temperature_2m_mean from the json file
    if 'elevation' in data:
        df['elevation'] = data['elevation']
    else:
        df['elevation'] = np.nan
    df['lat'] = lat_lon['lat'].iloc[i]
    df['lon'] = lat_lon['lon'].iloc[i]
    # concat the data
    result = pd.concat([result, df], axis=0)

100%|██████████| 1878/1878 [04:46<00:00,  6.55it/s]


In [42]:
# reset index
result.reset_index(drop=True, inplace=True)

In [57]:
df.columns

Index(['date', 'event_count_battles', 'event_count_explosions',
       'event_count_violence', 'fatalities_battles', 'fatalities_explosions',
       'fatalities_violence', 'GOSIF_GPP', 'rainfall_chirps', 'GOSIF_GPP_SD',
       'elevation', 'soil', 'lon', 'lat', 'temperature_2m_mean',
       'temperature_2m_mean_sd', 'shortwave_radiation_sum',
       'shortwave_radiation_sum_sd', 'precipitation_sum',
       'precipitation_sum_sd', 'nearest_neighbor_distance_x', 'market_lat',
       'market_lon', 'price_index', 'estimated_population', 'title',
       'overall_phase', 'country', 'phase3_worse_population',
       'phase3_worse_percentage', 'phase1_population', 'phase1_percent',
       'phase2_population', 'phase2_percent', 'phase3_population',
       'phase3_percent', 'phase4_population', 'phase4_percent',
       'phase5_population', 'phase5_percent', 'phase3_worse_percentage_manual',
       'phase3_plus_phase4', 'phase2_worse_percentage_manual',
       'phase2_plus_phase3', 'phase2_plus_p

In [59]:
# merge df and result on lat and lon, add indicator
new_df = df.merge(result, how='left', on=['lat', 'lon'], indicator=True)

In [60]:
# see merge result
print(new_df.groupby('_merge').size())

_merge
left_only         0
right_only        0
both          38776
dtype: int64


In [61]:
# drop _merge
new_df.drop('_merge', axis=1, inplace=True)

In [62]:
# save to csv
new_df.to_csv('merge_complete_elevation.csv', index=False)